Notes:
* Shuffle data
*transform and not fit_transform
*Where did train_target come from? in In [11]. I changed code to provide explicit reference at input
*What's the idea behind makign train_data 3x the size of dev_data? I know train is usually bigger but just wondering if you have a specific idea   

In [117]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt
import json
from pprint import pprint
import datetime
import pandas as pd

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *



In [67]:
train_dataset = 'train.json'
test_dataset = 'test.json'

In [68]:
def separate_data(dataset,data_type):
    data_features=[]
    data_msgtxt=[]
    data_titletxt=[]
    data_subreddits=[]
    data_ts=[]
    
    if data_type == 'train':
        data_target=[]
    
    '''
    for element in predict_dataset[0]:
        data_elements.append(element)
    '''
    
    for request in dataset:
        parts = []
        
        data_msgtxt.append(request['request_text_edit_aware'])
        data_titletxt.append(request['request_title'])
        data_subreddits.append(request['requester_subreddits_at_request'])
        
        month = datetime.datetime.fromtimestamp(request['unix_timestamp_of_request']).strftime("%m")
        day_of_month = datetime.datetime.fromtimestamp(request['unix_timestamp_of_request']).strftime("%d")
        hour = datetime.datetime.fromtimestamp(request['unix_timestamp_of_request']).strftime("%H")
        #rqst_ts = [int(month),int(day_of_month),int(hour)]
        rqst_ts = [int(hour)]
        
        if data_type == 'train':
            if request['requester_received_pizza'] == True:
                data_target.append(1)
            else:
                data_target.append(0)
        
        for element in data_elements:
            parts.append(request[element])
        
        data_features.append(parts)
        data_ts.append(rqst_ts)
    
    if data_type == 'train':
        return data_target, data_msgtxt, data_titletxt, data_subreddits, data_features, data_ts
    else:
        return data_msgtxt, data_titletxt, data_subreddits, data_features, data_ts

In [114]:
#Load data from JSON
train_data = json.loads(open(train_dataset).read())
test_data = json.loads(open(test_dataset).read())

#Row titles
data_elements=[u'requester_days_since_first_post_on_raop_at_request', u'requester_account_age_in_days_at_request', u'requester_number_of_posts_on_raop_at_request', u'requester_number_of_posts_at_request', u'requester_upvotes_plus_downvotes_at_request', u'requester_number_of_comments_at_request', u'requester_upvotes_minus_downvotes_at_request',u'requester_number_of_comments_in_raop_at_request',u'requester_number_of_subreddits_at_request']

# Set np seed
np.random.seed(0)

#Shuffle and split
np.random.shuffle(train_data) #list is mutable, this shuffles permanently
num_train = len(train_data)
train_raw_data=train_data[num_train/4:] #train is 3x bigger than dev
dev_raw_data=train_data[:num_train/4]
test_raw_data = test_data

train_target, train_msgtxt, train_titletxt, train_subreddits, train_features, train_ts = separate_data(train_raw_data,'train')
dev_target, dev_msgtxt, dev_titletxt, dev_subreddits, dev_features, dev_ts = separate_data(dev_raw_data, 'train')
test_msgtxt, test_titletxt, test_subreddits, test_features, test_ts = separate_data(test_raw_data, 'test')



print num_train
print len(train_target)
print len(test_msgtxt)

pprint(data_elements)
pprint(train_features[0])
pprint(train_ts[0])
pprint(train_target[0])

4040
3030
1631
[u'requester_days_since_first_post_on_raop_at_request',
 u'requester_account_age_in_days_at_request',
 u'requester_number_of_posts_on_raop_at_request',
 u'requester_number_of_posts_at_request',
 u'requester_upvotes_plus_downvotes_at_request',
 u'requester_number_of_comments_at_request',
 u'requester_upvotes_minus_downvotes_at_request',
 u'requester_number_of_comments_in_raop_at_request',
 u'requester_number_of_subreddits_at_request']
[0.0, 103.10547453703704, 0, 2, 24, 3, 18, 0, 3]
[9]
0


In [99]:
def vectorize(train_data,dev_data):
    # transform the train data
    vectorizer_train = CountVectorizer()
    #vectorizer_train = TfidfVectorizer()
    v_data_train = vectorizer_train.fit_transform(train_data)
    vocab_train = vectorizer_train.get_feature_names()
    
    '''
    # QUARANTINE - SEE ALTERNTE BELOW (cleaner)
    # transform the dev data using the same vocab
    vectorizer_dev = CountVectorizer(vocabulary=vocab_train)
    #vectorizer_dev = TfidfVectorizer(vocabulary=vocab_train)
    v_data_dev = vectorizer_dev.fit_transform(dev_data)
    '''
    # transform the dev data using the same vocab
    v_data_dev = vectorizer_train.transform(dev_data)     # 'transform' function will preserve previous vocab

    return v_data_train, v_data_dev, vocab_train

In [7]:
def vectorize_bigram(train_data,dev_data):
    # transform the train data
    vectorizer_train = CountVectorizer(ngram_range=(2,2))
    v_data_train = vectorizer_train.fit_transform(train_data)
    vocab_train = vectorizer_train.get_feature_names()
    # transform the dev data using the same vocab
    vectorizer_dev = CountVectorizer(ngram_range=(2,2),vocabulary=vocab_train)
    v_data_dev = vectorizer_dev.fit_transform(dev_data)
    return v_data_train, v_data_dev, vocab_train

In [8]:
# Get attributes using nnz and shape
def vectorizer_attrs(v_data):
    nonzero = v_data.nnz
    examples = v_data.shape[0]
    distinct_words = v_data.shape[1]
    avg_nonzero = float(nonzero)/examples
    total_entries = examples*distinct_words
    pct_nz_entries = float(nonzero)/total_entries*100
    return "Vocabulary size: " + str(distinct_words) + "\nAverage non-zero features per example: " + str(round(avg_nonzero,1)) + "\nFraction of non-zero entries in the matrix is " + str(nonzero) + "/" + str(total_entries) + " (" + str(round(pct_nz_entries,2)) + "%)"

In [100]:
def log_reg(train_data,train_label,dev_data):
    #lor = LogisticRegression(penalty='l1',C=1) #better, but save for later
    lor = LogisticRegression()
    lor.fit(train_data, train_label)
    lor_pred = lor.predict(dev_data)
    lor_pred_pr = lor.predict_proba(dev_data)
    allcoefs = lor.coef_.copy()
    # Return the prediction matrix, coefficients
    return lor_pred, lor_pred_pr, allcoefs

In [10]:
def get_topn(top_n,lorcoefs,vocab):
    allcoefs = lorcoefs.copy()
    lbls=allcoefs.shape[0]
    index=[]
    words=[]
    for num in range(top_n):
        mxindex = allcoefs.argmax(axis=1)
        for lbl in range(lbls):
            allcoefs[lbl][mxindex[lbl]] = 0
            index.append(mxindex[lbl])
            words.append(vocab[mxindex[lbl]])
    # With our new index of the top n words in each label, get the coefficient matrix of these words
    coefs=np.zeros((len(index),lbls))
    for lbl in range(lbls):
        for element in range(len(index)):
            coefs[element][lbl] = lorcoefs[lbl][index[element]]
    return words, coefs

In [108]:
def get_vectorized_logreg(train,train_target,test):
    train_vdata, dev_vdata, vocab = vectorize(train,test)
    prediction, predict_pr, allcoefs = log_reg(train_vdata,train_target,dev_vdata) #Where did train_target come from? I had to change funciton input to run on full data set
    words, coefs = get_topn(10,allcoefs, vocab)
    print vectorizer_attrs(train_vdata)
    
    return prediction #NOTE! I changed the return value to only return prediction. Sorry if this broke something!

In [113]:
#ACTUAL RUN CODE 

#Make prediction of dev data using msg text only. 
msg_pred = get_vectorized_logreg(train_msgtxt, train_target, dev_msgtxt)

#Make baseline model prediction that simply predicts the most common class (no pizza) at all times
baseline = [0]*len(msg_pred)

#Compare models
print "\nBaseline model (always predicts no pizza)"
print metrics.classification_report(dev_target,baseline)
print "\nSimple Logistic Regression model w/ Count Vectorizer, no regularization"
print metrics.classification_report(dev_target,msg_pred)

#Now make predictions on full dataset
msg_pred_test = get_vectorized_logreg(train_msgtxt+dev_msgtxt, train_target+dev_target, test_msgtxt)



Vocabulary size: 10576
Average non-zero features per example: 53.4
Fraction of non-zero entries in the matrix is 161880/32045280 (0.51%)

Baseline model (always predicts no pizza)
             precision    recall  f1-score   support

          0       0.76      1.00      0.86       765
          1       0.00      0.00      0.00       245

avg / total       0.57      0.76      0.65      1010


Simple Logistic Regression model w/ Count Vectorizer, no regularization
             precision    recall  f1-score   support

          0       0.78      0.85      0.81       765
          1       0.34      0.24      0.29       245

avg / total       0.67      0.70      0.68      1010

Vocabulary size: 12317
Average non-zero features per example: 53.6
Fraction of non-zero entries in the matrix is 216394/49760680 (0.43%)


1631

In [134]:
#FAIRLY RIDICULOUS WORKFLOW- I RELOAD THE TEST DATA IN PANDAS
'''
This cell outputs test data to format for submission to Kaggle
'''


###LOAD IN DATA
with open('test.json','r') as fp: 
    json_data = json.load(fp)

df = pd.io.json.json_normalize(json_data)

#extract request_id so we can match against predictions for submission to kaggle
req = df['request_id']
##print req

#make prediction in previous cell into a pandas series
test_pred_series = pd.Series(msg_pred_test,name="requester_received_pizza")
##print test_pred_series

#now join into data frame
out = pd.concat([req,test_pred_series], axis=1)
print out

#write data frame to csv (using kaggles sample submission csv for format)
out.to_csv('submit_to_kaggle.csv',index=False)


     request_id  requester_received_pizza
0      t3_i8iy4                         0
1     t3_1mfqi0                         0
2      t3_lclka                         0
3     t3_1jdgdj                         0
4      t3_t2qt4                         0
5      t3_pvojb                         0
6     t3_142n4c                         0
7     t3_17rja6                         0
8     t3_1lg6u2                         1
9     t3_1b0mtx                         0
10     t3_mmivq                         0
11    t3_11ux2a                         0
12     t3_klige                         0
13    t3_1iv3cj                         0
14    t3_13dh0w                         0
15     t3_li326                         0
16     t3_xww8y                         0
17     t3_kogen                         1
18     t3_ik8w1                         0
19     t3_j3nnl                         0
20     t3_j6xvd                         0
21    t3_1lsrj5                         0
22     t3_idld7                   

In [134]:
title_prob = get_vectorized_logreg(train_titletxt, dev_titletxt)

Vocabulary size: 3832
Average non-zero features per example: 11.3
Fraction of non-zero entries in the matrix is 34173/11610960 (0.29%)


1010


1010


[0.0, 0.0, 0, 0, 0, 0, 0, 0, 0]
['10', '05', '18']


In [171]:
feature_prediction, feature_predict_pr, feature_allcoefs = log_reg(train_features,train_target,dev_features)
ts_prediction, ts_predict_pr, ts_allcoefs = log_reg(train_ts,train_target,dev_ts)
print feature_predict_pr
print ts_predict_pr

[[ 0.79120797  0.20879203]
 [ 0.76458161  0.23541839]
 [ 0.79120797  0.20879203]
 ..., 
 [ 0.77608364  0.22391636]
 [ 0.78600322  0.21399678]
 [ 0.78588184  0.21411816]]
[[ 0.75388208  0.24611792]
 [ 0.78420871  0.21579129]
 [ 0.76309099  0.23690901]
 ..., 
 [ 0.75574299  0.24425701]
 [ 0.75759436  0.24240564]
 [ 0.75943616  0.24056384]]


In [221]:
prediction_array=[]
predict_score=[]
score_val=0.1
prediction=[]
for i in range(len(dev_features)):
    proba=[]
    proba.append(msg_prob[i])
    proba.append(title_prob[i])
    proba.append(feature_predict_pr[i,1])
    proba.append(ts_predict_pr[i,1])
    prediction_array.append(proba)
    predict_score.append(sum(proba))
    if sum(proba)>=score_val:
        prediction.append(1)
    else:
        prediction.append(0)

In [222]:
print "F1 score of final model is " + str(round(metrics.f1_score(dev_target,prediction),3))

F1 score of final model is 0.413


In [225]:
print prediction
print dev_target

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 